In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# speech content decode
import pickle
import os
import numpy as np
import pandas as pd 
from plotting_utils import plot_fncs, color_pals

In [ ]:
data_path = '../data'

In [ ]:
import numpy as np
import seaborn as sns
import scipy.stats as stat_package
from sklearn.metrics import confusion_matrix
from scipy.stats import sem
import logging
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
with open(f'{data_path}/model_ablation_REVISE.pkl','rb') as f:
    df = pickle.load(f)

In [ ]:
with open(f'{data_path}/read_dfs.pkl','rb') as f:
    df_read = pickle.load(f)

In [ ]:
with open(f'{data_path}/window_sals.pkl','rb') as f:
    window_sals = pickle.load(f)

In [ ]:
with open(f'{data_path}/sample_read_erps.pkl','rb') as f:
    erps = pickle.load(f)

In [ ]:
from scipy.ndimage import gaussian_filter1d
erps = [gaussian_filter1d(erp,4,axis=1) for erp in erps]

In [ ]:
with open(f'{data_path}/function_resp_df.pkl','rb') as f:
    df_activations = pickle.load(f)

In [ ]:
# set plot params 
import warnings
warnings.filterwarnings('ignore')
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
# mpl.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams.update({'font.size': 6})
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# from gimutil.configuration import config
# from gimutil.visualization import plotting_tools
import matplotlib.pyplot as plt
lw = 0.5
    
        
# Computes and plots the dendrogram
import seaborn as sns
import matplotlib.font_manager
import matplotlib as mpl
from statannot import add_stat_annotation

region_pal = color_pals.regions
region_pal_dict = color_pals.region_dict
b3_gate_pal = region_pal.copy()
b3_gate_pal = b3_gate_pal[:3]
b3_gate_pal.append('#4F4E4C')
b3_gate_order=['temporal','precentral','postcentral','all']

b1_gate_pal = region_pal.copy()
b1_gate_pal = list(np.array(b1_gate_pal)[[1,2,3]])
b1_gate_pal.append('#4F4E4C')
b1_gate_order =['precentral','postcentral','frontal','all']


distractor_pal = list(np.array(color_pals.distractors)[[0,-1]])
distractor_order = ['Attempted speech','Listen']
base_pal = sns.color_palette('deep').as_hex()
palette = ['#4F4E4C',base_pal[0],base_pal[1],base_pal[2]]

# Specifies plot parameters
rows = {
    'third_start' : 0,
    'third_stop' : 150,
    'fourth_start' : 250,
    'fourth_stop' : 400,
    'fifth_start' : 500,
    'fifth_stop' : 700,
    'total'      : 700,
}

cols = {
    'erp1_start' : 0,
    'erp1_stop' : 150,
    'erp2_start' : 160,#200,
    'erp2_stop' : 310,
    'erp3_start' : 320,
    'erp3_stop' : 470,
    'salsquant_start' : 530,
    'salsquant_stop' : 800,
    'readquant1_start': 400,
    'readquant1_stop' : 550,
    'readquant2_start' : 650,
    'readquant2_stop' : 800,
    'ablation_start' : 100,
    'ablation_stop' : 700,
    'total'      : 800
}
all_panel_params = {
    'erp1' : {'row_and_col_spec' : ('third', 'erp1')},
    'erp2' : {'row_and_col_spec' : ('third', 'erp2')},
    'readquant1' : {'row_and_col_spec' : ('third', 'readquant1')},
    'readquant2' : {'row_and_col_spec' : ('third', 'readquant2')},
    'early_sal' : {'row_and_col_spec' : ('fourth', 'erp1')},
    'mid_sal' : {'row_and_col_spec' : ('fourth', 'erp2')},
    'late_sal' : {'row_and_col_spec' : ('fourth', 'erp3')},
    'readingsal' : {'row_and_col_spec' : ('fourth', 'salsquant')},
    'ablation' : {'row_and_col_spec' : ('fifth', 'ablation')},
    
    
}

dims = np.array([cols['total'],rows['total']])/np.max([rows['total'],cols['total']])
scale = 8.17
# Creates the plot
fig, axs = plot_fncs.setup_figure(
    all_panel_params=all_panel_params, row_specs=rows, col_specs=cols,
    figsize=scale*dims #15,12
)


alt_pal = sns.color_palette('bone_r').as_hex()
alt_pal = [alt_pal[-1],alt_pal[3],alt_pal[0]]

ax = axs['erp1']
ax = plot_fncs.plot_erps(erps,0, np.linspace(-2, 2, num=erps[0].shape[1]), ax,do_axes=True,colors=alt_pal,legends=None)    
ax.set_title('Strong language pref.')
sns.despine(ax=ax,offset={'left': 5,'bottom': 5})
ax.set_xlim([-2,2])
ax = axs['erp2']
ax = plot_fncs.plot_erps(erps,1, np.linspace(-2, 2, num=erps[0].shape[1]), ax,do_axes=True,colors=alt_pal,
                         legends=['Words (ex: India)','Sentences (ex: Nobody likes snakes)','False fonts (ex: ƋΨΞƩΔ)'])    
ax.set(yticks=[],ylabel='')   
ax.spines['left'].set_visible(False)
ax.set_title('Weak language pref.')
ax.set_xlim([-2,2])
sns.despine(ax=ax,offset={'bottom': 5},left=True)
ax.legend(fontsize=6,frameon=False)

### PANEL: Quantify reading against false
ax = axs['readquant1']

ax_lim = [-0.2,0.5]
ax_ticks = [-0.2,0,0.5]
g = sns.scatterplot(ax=ax,data=df_read[(df_read.p < 0.05) & (df_read.read_max >0)],y='read_max',x='read_false',clip_on=False,
               color=palette[0],alpha=0.4,style='patient',s=15)
g.legend(frameon=False)
ax.set(ylim=ax_lim,xlim=ax_lim,xticks=ax_ticks,yticks=ax_ticks,
       ylabel='HGA reading words',xlabel='HGA reading false strings')
ax.plot(ax_lim,ax_lim,color='k',linestyle='--')
sns.despine(ax=ax,offset=5)

from scipy.stats import wilcoxon
print(wilcoxon(df_read[df_read.p < 0.05].read_max.values,df_read[df_read.p < 0.05].read_false.values))

### PANEL: Quantify reading against sentences
ax = axs['readquant2']
ax_lim = [0,0.8]
ax_ticks = [0,0.4,0.8]
sns.scatterplot(ax=ax,data=df_read[(df_read.p < 0.05) & (df_read.read_max >0)],y='read_sent',x='read_max',clip_on=False,
               color=palette[0],alpha=0.4,style='patient',legend=None,s=15)
ax.set(ylim=ax_lim,xlim=ax_lim,xticks=ax_ticks,yticks=ax_ticks,
       xlabel='HGA reading words',ylabel='HGA reading sentences')
ax.plot(ax_lim,ax_lim,color='k',linestyle='--')
sns.despine(ax=ax,offset=5)
print(wilcoxon(df_read[df_read.p < 0.05].read_sent.values,df_read[df_read.p < 0.05].read_max.values))



with open(f'{data_path}/b3_electrode_anatomical_localizations.pkl','rb') as f:
    anat = pickle.load(f)
anat[(anat == 'superiortemporal') | (anat == 'middletemporal')] = 'temporal lobe'
anat[(anat == 'caudalmiddlefrontal') | (anat == 'parsopercularis')] = 'precentral'
anat[(anat == 'supramarginal')] = 'postcentral' 

tops = []

b3_xlims = [205,515] #310
b3_ylims = [100,525] #425

### PANEL: Brain plots of early, mid, late sals
max_size = 20
ax = axs['early_sal'] 
plot_fncs.plot_vals_on_brain(window_sals[0],palette[0],fig,ax,data_path,size_max=max_size,size_min=3.,
                            region_color=region_pal_dict['precentral'])
ax.set(title='Pre go-cue')
ax.set(xlim=b3_xlims,ylim=b3_ylims)

ax = axs['mid_sal'] 
plot_fncs.plot_vals_on_brain(window_sals[1],palette[0],fig,ax,data_path,size_max=max_size,size_min=3.,
                            region_color=region_pal_dict['precentral'])
ax.set(title='Centered on go-cue')
ax.set(xlim=b3_xlims,ylim=b3_ylims)

ax = axs['late_sal'] 
plot_fncs.plot_vals_on_brain(window_sals[2],palette[0],fig,ax,data_path,size_max=max_size,size_min=3.,
                            region_color=region_pal_dict['precentral'])
ax.set(title='After go-cue')
ax.set(xlim=b3_xlims,ylim=b3_ylims)

d_sals_time = {'sal':[],'region':[],'time':[]}
d_sals_time['sal'].extend(window_sals[0])
d_sals_time['sal'].extend(window_sals[1])
d_sals_time['sal'].extend(window_sals[2])

d_sals_time['region'].extend(anat)
d_sals_time['region'].extend(anat)
d_sals_time['region'].extend(anat)

d_sals_time['time'].extend(np.repeat('Pre go-cue',253))
d_sals_time['time'].extend(np.repeat('Centered \non go-cue',253))
d_sals_time['time'].extend(np.repeat('After go-cue',253))


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
norm_0 = scaler.fit_transform(np.array(window_sals[0]).reshape(-1,1)).squeeze()
norm_1 = scaler.fit_transform(np.array(window_sals[1]).reshape(-1,1)).squeeze()
norm_2 = scaler.fit_transform(np.array(window_sals[2]).reshape(-1,1)).squeeze()


d_sals_time = {'sal_1':window_sals[0],'sal_2': window_sals[1], 'sal_3': window_sals[2],
               'region':anat}
d_sals_time = {'sal_1':norm_0,'sal_2': norm_1, 'sal_3': norm_2,
               'region':anat}
d_sals_time = pd.DataFrame(d_sals_time)

perc_cut = 90
keep = ( (d_sals_time.sal_1 >= np.percentile(d_sals_time.sal_1,perc_cut)) | (d_sals_time.sal_2 >= np.percentile(d_sals_time.sal_2,perc_cut))
        | (d_sals_time.sal_3 >= np.percentile(d_sals_time.sal_3,perc_cut)) )

d_sals_time = d_sals_time[keep]


### PANEL: Quant of the saliences over time 
ax = axs['readingsal']
import matplotlib.pyplot as plt
panel_f = pd.read_excel(f'{data_path}/fig5_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_f').iloc[:,1:]

# define the electrodes corresponding to each cluster 
hand_clust = plot_fncs.adjust_elec_inds(panel_f['top_hand']).values
lips_clust = plot_fncs.adjust_elec_inds(panel_f['top_lips']).values
coronal_clust = plot_fncs.adjust_elec_inds(panel_f['top_coronal']).values

artic = []
p_val = [] 
for i, row in d_sals_time.iterrows():
    artic.append( i in lips_clust or i in hand_clust or i in coronal_clust ) 
    percep_stats = df_activations.iloc[i][['read_p','listen_p']]
    p_val.append(percep_stats[0] < 0.05 and percep_stats[1] < 0.05)

    
d_sals_time['artic'] = artic
d_sals_time['read_listen'] = p_val
sal_vals = np.concatenate( (d_sals_time.sal_1,d_sals_time.sal_2,d_sals_time.sal_3))
artic = np.concatenate( (d_sals_time.artic,d_sals_time.artic,d_sals_time.artic) ) 
read_listen = np.concatenate( (d_sals_time.read_listen,d_sals_time.read_listen,
                               d_sals_time.read_listen) ) 
time_win = np.concatenate( (np.repeat('Pre-go cue',len(d_sals_time.sal_1)),
                            np.repeat('Centered on go cue',len(d_sals_time.sal_2)),
                            np.repeat('After-go cue',len(d_sals_time.sal_3)) ) )

tag = artic.copy()
tag[artic] = 'Artic.'
tag[read_listen] = 'Multiplexed'

d_sals_time_plt = pd.DataFrame({'Time window': time_win, 'Salience': sal_vals,
                                'Artic': artic, 'Multiplexed': read_listen})
tag = []
for i,row in d_sals_time_plt.iterrows():
    if(row.Artic):
        tag.append('Artic.')
    elif(row.Multiplexed):
        tag.append('Multiplexed')
    else:
        tag.append('None')
        
d_sals_time_plt['Category'] = tag

from scipy.stats import ranksums
p_val = []
box_pairs = []
for x_align in ['Pre-go cue','Centered on go cue','After-go cue']:
    p_val.append(ranksums(d_sals_time_plt[(d_sals_time_plt['Time window'] == x_align) 
                             & (d_sals_time_plt['Category'] == 'Artic.')].Salience,
             d_sals_time_plt[(d_sals_time_plt['Time window'] == x_align) 
                             & (d_sals_time_plt['Category'] == 'Multiplexed')].Salience )[1])
    box_pairs.append(  ((x_align, 'Multiplexed'), (x_align, 'Artic.')) )

for cat in ['Shared','Artic.']:
    p_val.append(ranksums(d_sals_time_plt[(d_sals_time_plt['Time window'] == 'Pre-go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience,
             d_sals_time_plt[(d_sals_time_plt['Time window'] == 'Centered on go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience )[1])
    box_pairs.append(  (('Pre-go cue', cat), ('Centered on go cue', cat)) )    

    p_val.append(ranksums(d_sals_time_plt[(d_sals_time_plt['Time window'] == 'Pre-go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience,
             d_sals_time_plt[(d_sals_time_plt['Time window'] == 'After-go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience )[1])
    box_pairs.append(  (('Pre-go cue', cat), ('After-go cue', cat)) )    
    
    p_val.append(ranksums(d_sals_time_plt[(d_sals_time_plt['Time window'] == 'After-go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience,
             d_sals_time_plt[(d_sals_time_plt['Time window'] == 'Centered on go cue') 
                             & (d_sals_time_plt['Category'] == cat)].Salience )[1])
    box_pairs.append(  (('After-go cue', cat), ('Centered on go cue', cat)) )    

from statsmodels.stats.multitest import multipletests
p_val = multipletests(p_val,method='holm')[1]
print(len(p_val))
box_pairs = [b for b,p in zip(box_pairs,p_val) if p < 0.05]
p_val = [p for p in p_val if p < 0.05]



import seaborn as sns

pal = []
pal.append(sns.color_palette('Set2').as_hex()[0])
pal.append(sns.color_palette('Set2').as_hex()[2])

g = sns.boxplot(ax=ax,data=d_sals_time_plt, x='Time window', y='Salience', hue='Category',
           hue_order=['Multiplexed','Artic.'],palette=pal,boxprops=dict(alpha=0.8),linewidth=lw) #'binary'
add_stat_annotation(ax, data=d_sals_time_plt,x='Time window', y='Salience', hue='Category',
                    box_pairs=box_pairs, hue_order=['Multiplexed','Artic.'],
                    test=None, loc='outside', verbose=2,pvalues=p_val,perform_stat_test=False,
                    text_offset=0.05,line_offset_to_box=0)


ax.set(xticks=[0,1,2],xticklabels=['Pre go-cue','Centered \non go-cue','After go-cue'],ylim=[0,1],yticks=[0,0.5,1])
ax.set(ylabel='Elec. contribution (A.U.)',xlabel='')

# ax.legend(bbox_to_anchor=(0.4,0.7),ncol=1,frameon=False)
h, _ = ax.get_legend_handles_labels()
ax.legend(h, ['Shared', 'Artic.'], bbox_to_anchor=(0.4,0.7),ncol=1,frameon=False);
    
sns.despine(ax=ax,offset=5,bottom=True)
# ax.spines['bottom'].set_visible(False)
ax.xaxis.set_tick_params(length=0)
plt.setp(ax.collections, clip_on=False)
plt.setp(ax.lines, clip_on=False, zorder=5)

###########

ax = axs['ablation']
wins = ['200.0_600.0','400.0_800.0','600.0_1000.0']
model_names = ['Pre go-cue', 'Centered on go-cue','After go-cue']
from scipy.stats import ranksums 
ps = []
box_pairs = []
for win in model_names:
    df_ = df[df.win == win]
    ps.append(ranksums(df_[df_.model == 'All elecs.'].acc,df_[df_.model == 'No artic. elecs.'].acc).pvalue)
    box_pairs.append( ((win,'All elecs.'), (win,'No artic. elecs.')))
    
    ps.append(ranksums(df_[df_.model == 'All elecs.'].acc,df_[df_.model == 'No multiplexed elecs.'].acc).pvalue)
    box_pairs.append( ((win,'All elecs.'), (win,'No multiplexed elecs.')))

    ps.append(ranksums(df_[df_.model == 'No multiplexed elecs.'].acc,df_[df_.model == 'No artic. elecs.'].acc).pvalue)
    box_pairs.append( ((win,'No artic. elecs.'), (win,'No multiplexed elecs.')))


    
    
from statsmodels.stats.multitest import multipletests
ps = multipletests(ps,method='holm')[1]
new_ps, new_bps = [],[]
for p,b in zip(ps,box_pairs):
    if p < 0.05:
        new_ps.append(p)
        new_bps.append(b)

import seaborn as sns
from statannot import add_stat_annotation

pal = [sns.color_palette('binary')[3]]
pal.append(sns.color_palette('Set2').as_hex()[0])
pal.append(sns.color_palette('Set2').as_hex()[2])

sns.boxplot(data=df,x='win',hue='model',y='acc',ax=ax,palette=pal,linewidth=lw)


add_stat_annotation(ax, data=df,x='win',hue='model',y='acc',
                    box_pairs=new_bps, hue_order=['All elecs.','No multiplexed elecs.',
                                                    'No artic. elecs.'],
                    test=None, loc='outside', verbose=2,pvalues=new_ps,perform_stat_test=False)
ax.axhline(10,color='k',linestyle='--')

# ax.legend(frameon=False,title='')
h, _ = ax.get_legend_handles_labels()
ax.legend(h, ['All elecs.','No shared elecs.','No artic. elecs.'], bbox_to_anchor=(0.4,0.7),ncol=1,frameon=False);

ax.set(xlabel='',ylabel='Classification accuracy (%)',yticks=[0,50,100],ylim=[0,100])
sns.despine(ax=ax,bottom=True,offset=5)
ax.xaxis.set_tick_params(length=0)


pad_top = 100
axs['erp1'].annotate('A',(-40, pad_top),xycoords='axes points',weight='bold', ha='right',fontsize=9)
axs['readquant1'].annotate('B',(-40, pad_top),xycoords='axes points',weight='bold', ha='right',fontsize=9)
axs['readquant2'].annotate('C',(-40, pad_top),xycoords='axes points',weight='bold', ha='right',fontsize=9)

axs['early_sal'].annotate('D',(-35, pad_top),xycoords='axes points',weight='bold', ha='right',fontsize=9)

axs['early_sal'].annotate('Attempted speech \n contributions', xy=(-0.2, 0.5), rotation=90, 
            va='center', ha='center',xycoords='axes fraction',fontsize=6)


axs['readingsal'].annotate('E',(-35, pad_top),xycoords='axes points',weight='bold', ha='right',fontsize=9)
axs['ablation'].annotate('F',(-40, pad_top+25),xycoords='axes points',weight='bold', ha='right',fontsize=9)


plt.show()